In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import os,sys,inspect

current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

import ipywidgets as widgets
from IPython.display import display

In [2]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
from db.models import TickerReturn, Ticker
from db.configuration import database_connection

In [4]:
import pendulum
start_period = pendulum.naive(2019, 10, 1)

In [62]:
tickers_list = ["SHOP", "AC.TO", "MU", "PHM", "ENB", "IBKR", 
                "MSFT", "AAPL", "SPY", "TSLA", "SHOP.TO", "BABA", "PFE"]
tickers_list = ["T.TO", "BB", "ATD-B.TO", "REAL.TO", "QTRH.TO", "DRM.TO", "MX.TO", "KEY.TO", "FTT.TO", "BAM-A.TO", "BYD.TO", 
                "L", "ADSK", "FSLY", "ANET", "ASML", "BERY", "BWA", "SCHW", "FLGT", "WAL", "TWTR", "SHOP"]
exlist = tickers_list

In [63]:
tickers = Ticker.select().where(Ticker.ticker.in_(exlist))
base_query = TickerReturn.select(
    TickerReturn.datetime,
    TickerReturn.close,
).where(
    TickerReturn.interval == '1d',
    TickerReturn.datetime > start_period
).order_by(
    TickerReturn.datetime.asc()
).join(Ticker)

In [64]:
dataframes = []
for ticker in exlist:
    ticker_query = base_query.where(TickerReturn.ticker == Ticker.get(Ticker.ticker == ticker))
    dataframe = pd.read_sql(ticker_query.sql()[0], database_connection(),
        params=ticker_query.sql()[1],
        index_col='datetime'
    )
    df = pd.DataFrame({ticker: dataframe.close}, index=dataframe.index)
    print(ticker)
    print(df.shape)
    dataframes.append(df)
dataframes = pd.concat(dataframes, axis=1) #.dropna()

T.TO
(311, 1)
BB
(312, 1)
ATD-B.TO
(310, 1)
REAL.TO
(311, 1)
QTRH.TO
(311, 1)
DRM.TO
(311, 1)
MX.TO
(311, 1)
KEY.TO
(311, 1)
FTT.TO
(311, 1)
BAM-A.TO
(311, 1)
BYD.TO
(311, 1)
L
(312, 1)
ADSK
(312, 1)
FSLY
(312, 1)
ANET
(312, 1)
ASML
(312, 1)
BERY
(312, 1)
BWA
(312, 1)
SCHW
(312, 1)
FLGT
(312, 1)
WAL
(312, 1)
TWTR
(312, 1)
SHOP
(312, 1)


In [65]:
ticker_returns = dataframes.pct_change().dropna()

In [66]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [67]:
mu = expected_returns.return_model(dataframes, method="capm_return")
S = risk_models.risk_matrix(dataframes, method="oracle_approximating")

In [68]:
ef = EfficientFrontier(mu, S)

In [69]:
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

In [70]:
cleaned_weights

OrderedDict([('T.TO', 0.04134),
             ('BB', 0.04426),
             ('ATD-B.TO', 0.04257),
             ('REAL.TO', 0.04386),
             ('QTRH.TO', 0.04237),
             ('DRM.TO', 0.04257),
             ('MX.TO', 0.04421),
             ('KEY.TO', 0.04473),
             ('FTT.TO', 0.04291),
             ('BAM-A.TO', 0.04507),
             ('BYD.TO', 0.0423),
             ('L', 0.04385),
             ('ADSK', 0.0442),
             ('FSLY', 0.04354),
             ('ANET', 0.04336),
             ('ASML', 0.04321),
             ('BERY', 0.0428),
             ('BWA', 0.04271),
             ('SCHW', 0.04397),
             ('FLGT', 0.04404),
             ('WAL', 0.04422),
             ('TWTR', 0.04389),
             ('SHOP', 0.04407)])

In [71]:
ef.portfolio_performance(verbose=True)

Expected annual return: 57.0%
Annual volatility: 34.1%
Sharpe Ratio: 1.61


(0.57004789912454, 0.3409679778846534, 1.6131951819552293)

In [77]:
from pypfopt import DiscreteAllocation

da = DiscreteAllocation(cleaned_weights, dataframes.iloc[-1], total_portfolio_value=10000)
alloc, leftover = da.lp_portfolio()
print(f"Leftover: ${leftover:.2f}")
pd.Series(alloc).sort_index()

Leftover: $5.24


ADSK          2
ANET          2
ASML          1
ATD-B.TO     10
BAM-A.TO      9
BB           63
BERY          8
BWA          11
BYD.TO        2
DRM.TO       20
FLGT         10
FSLY          5
FTT.TO       16
KEY.TO       20
L            10
MX.TO         7
QTRH.TO     168
REAL.TO      22
SCHW          8
T.TO         16
TWTR          8
WAL           7
dtype: int64